In [13]:
%run crypt_backend.ipynb

# ASCII Encoding 
## Example 2.5
We implement 'conv_to_ASCII' and 'ASCII_to_string' on the message 'Hello world!'\
Then perform binary conversion. Note that in **Example 2.5** we manually extend the binary code to 8 bits where the leading bit is used for parity.

In [20]:
ascii_message=conv_to_ASCII('Hello world!')
print(ascii_message)

[72, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 33]


In [24]:
for char in ascii_message:
    print(bin(char)[2:])

1001000
1100101
1101100
1101100
1101111
100000
1110111
1101111
1110010
1101100
1100100
100001


# Encrypting a Message
## Example 2.7
We encrypt the message 'Hello world!' using 'fast_encrypt'.\
We use the public key $(e,n)=(17,1363)$.\
Note that the ASCII conversion takes place within the function.

In [31]:
cypher_text=fast_encrypt('Hello world!',17,1363)

# Decrypting a Message
## Example 2.9
We decrypt the encrypted message obtained in **Example 2.7**.

In [38]:
plain_text=fast_decrypt(cypher_text,29,47,985)

In [40]:
ASCII_to_string(plain_text)

'Hello world!'

# Factorisation Attack on 'RSA-encrypted-1.txt'
Use SFA to factorise the modulus in the public key $N=99157$

In [132]:
shores_factorise(99157)

(229, 433)

Calculate private keys using obtained $(p,q)=(229,433)$ and $e=289$.

In [138]:
phi_n=(229-1)*(433-1)
d=modular_inverse(289,phi_n)

Reads encrypted message into a list.

In [143]:
def read_integers_from_file(filename):
    with open(filename, 'r') as file:
        return [int(line.strip()) for line in file if line.strip().isdigit()]

Decryptes message using 'fast_decrypt'.

In [149]:
encrypted_list=read_integers_from_file('RSA-encrypted-1.txt')
decrypted_lst=(fast_decrypt(encrypted_list,229,433,d))

Converts message from ASCII to string. Note that '\n' is the ASCII control for a newline.

In [155]:
ASCII_to_string(decrypted_lst)

'The fundamental problem of communication is that of reproducing at one point either exactly or ap-\nproximately a message selected at another point. Frequently the messages have meaning; that is they refer\nto or are correlated according to some system with certain physical or conceptual entities. These semantic\naspects of communication are irrelevant to the engineering problem. The significant aspect is that the actual\nmessage is one selected from a set of possible messages. The system must be designed to operate for each\npossible selection, not just the one which will actually be chosen since this is unknown at the time of design. (From C. Shannon, A Mathematical Theory of Communication, The Bell System Technical Journal,\nVol. 27, pp. 379–423, 623–656, July, October, 1948)'

# Benchmarking Functions
For benchmarking we define functions that randomly generate numbers of determined bit length.\
We also need a function that can generate a number coprime to a given $n$.

In [181]:
import secrets
import random
def generate_random_2048_bit_number():
    return secrets.randbits(2048)
def generate_random_1024_bit_number():
    return secrets.randbits(1024)
def generate_random_8_bit_number():
    return secrets.randbits(8)
    
def generate_coprime(n):
    """Generates a random 1024-bit number that is coprime to n."""
    bit_size=1024
    while True:
        rand_num = secrets.randbits(bit_size) | 1  # Ensure it's odd
        if scaled_gcd(rand_num, n) == 1:  # Check gcd(rand_num, n) == 1
            return rand_num

# Encryption Benchmarking
RSA encryption technique comparison performing $m^e\mod n$ with a random 8 bit $m$ (ASCII is 8 bit), random 2048 bit $n$ (NIST standard public modulus) and $e=65537$ (commonly used public exponent). Note that of course, results depend on the machine and jupyters %timeit function will automatically limit the amount of runs unless specified.

In [166]:
%timeit (generate_random_8_bit_number()**65537)%generate_random_2048_bit_number()

11.5 ms ± 480 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [169]:
%timeit mod_exponentiate(generate_random_8_bit_number(),65537,generate_random_2048_bit_number())

102 μs ± 2.52 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [183]:
def benchmark2():
    a=generate_random_1024_bit_number()
    fast_decrypt([generate_random_8_bit_number()],a,generate_coprime(a),65537)

%timeit benchmark2()

482 μs ± 30.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Decryption Benchmarking
RSA decryption technique comparison performing $c^d\mod n$ with a random 2048 bit $c$ (standard encrypted word size using 2048 bit modulus), random 2048 bit $n$ (NIST standard public modulus) and a random 2048 bit $d$ (standard private key size). Note that of course, results depend on the machine and jupyters %timeit function will automatically limit the amount of runs unless specified.

In [191]:
%timeit naive_decrypt([generate_random_2048_bit_number()],generate_random_2048_bit_number(),generate_random_2048_bit_number())

39.9 ms ± 947 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [189]:
def benchmark():
    a=generate_random_1024_bit_number()
    fast_decrypt([generate_random_2048_bit_number()],a,generate_coprime(a),generate_random_2048_bit_number())

%timeit benchmark()

13.4 ms ± 562 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
